In [1]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display

from utils import *
import skin_seg as ss

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))


Running on device: cuda:0


In [3]:
mtcnn = MTCNN(keep_all=True, device=device)

model = ss.ConvAutoEncoderArrayDecoder(num_decoders=1)
model.to(device)
model.load_state_dict(torch.load("models/skin_seg"))
model.eval()

ConvAutoEncoderArrayDecoder(
  (encoder): ConvAutoEncoder(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (batchNorm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batchNorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batchNorm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batchNorm4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batchNorm5): BatchNorm2d(512, eps=1e-05, momentum=0.1, a

In [4]:
# video = mmcv.VideoReader('video.mp4')
# frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

# display.Video('video.mp4', width=640)

# define a video capture object
vid = cv2.VideoCapture(0)
face_not_visible = True
  
while(True):
      
    # Capture the video frame by frame
    ret, frame = vid.read()

    # Detect faces
    frame = Image.fromarray(frame)
    boxes, probs, landmarks = mtcnn.detect(frame, landmarks=True)

    # Draw faces
    frame_draw = frame.copy()
    draw = ImageDraw.Draw(frame_draw)
    #Check if net found faces
    if(boxes is not None):
        face_not_visible = False
        for box, landmark in zip(boxes, landmarks):
            draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
            draw.polygon(landmarks, outline=(255,0,0))
            # print(landmark.shape)
            # print(landmark)
    else:
        face_not_visible = True
        print("Face not visible!")

    idx = 0
    if(boxes is not None):
        np_frame = np.asarray(frame)

        area, idx = calc_area(boxes) # return area and idx of largest box (small filter if there are multiple people)
        box = boxes[idx]

        # calculate center of largest box and get bounding box
        squares = calc_square_box(box, img_dim = np_frame.shape)

        # extract 
        square_seg = np_frame[squares[3]:squares[1], squares[2]:squares[0]]
        square_frame_original = cv2.resize(square_seg, (64,64), interpolation = cv2.INTER_AREA)

        #Scaling for 64x64 model input
        square_frame = np.transpose(square_frame_original, (2,0,1)) / 256
        square_frame = np.reshape(square_frame, newshape=(1,) + square_frame.shape)
        square_img = torch.tensor(square_frame, dtype=torch.float32)
        square_img = square_img.to(device)

        #Get Mask
        masks, _ = model(square_img)
        
        masks = masks[0].cpu().detach().numpy()
        mask = np.transpose(masks[0], (1,2,0))
        mask_reshape = cv2.resize(mask, (square_seg.shape[1], square_seg.shape[0]))
        mask = mask_reshape > 0.6 #mask
        mask = np.reshape(mask, mask.shape + (1,))
        img_masked = square_seg * mask #mask overlayed on image

        cv2.imshow('img_crop', square_seg)
        cv2.imshow('mask', mask_reshape)
        cv2.imshow('mask on image', img_masked)

        draw.rectangle(squares.tolist(), outline=(0, 255, 0), width=6)



    # Display the resulting frame
    cv2.imshow('frame_draw', np.asarray(frame_draw))
    # the 'q' button is set as quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

Face not visible!
Face not visible!
Face not visible!
Face not visible!
Face not visible!
Face not visible!
Face not visible!
Face not visible!


In [5]:
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

In [6]:
squares[0]
# np_fram[int(squares[0]):int(squares[2])]

468